In [1]:
import boto3
import json
import os
import joblib
import pickle
import tarfile
import sagemaker
from sagemaker.estimator import Estimator
import time
from time import gmtime, strftime
import subprocess

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
#Setup
client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")
boto_session = boto3.session.Session()
s3 = boto_session.resource('s3')
region = boto_session.region_name
print(region)
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

us-east-1


In [3]:
#Build tar file with model data + inference code
bashCommand = "tar -cvpzf model.tar.gz random_forest_model.pkl inference.py"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

In [4]:
# retrieve sklearn image
image_uri = sagemaker.image_uris.retrieve(
    framework="sklearn",
    region=region,
    version="1.2-1",
    py_version="py3",
    instance_type="ml.t3.medium",
)

#Bucket for model artifacts
default_bucket = sagemaker_session.default_bucket()
print(default_bucket)

#Upload tar.gz to bucket
model_artifacts = f"s3://{default_bucket}/model.tar.gz"
evaluation_metrics_uri = f"s3://{default_bucket}/evaluation_metrics.json"
response = s3.meta.client.upload_file('model.tar.gz', default_bucket, 'model.tar.gz')
response1 = s3.meta.client.upload_file('evaluation_metrics.json', default_bucket, 'evaluation_metrics.json')
print(evaluation_metrics_uri)

sagemaker-us-east-1-767397996001
s3://sagemaker-us-east-1-767397996001/evaluation_metrics.json


In [5]:
#Step 1: Model Creation
model_name = "sklearn-test" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)
create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": model_artifacts,
            "Environment": {'SAGEMAKER_SUBMIT_DIRECTORY': model_artifacts,
                           'SAGEMAKER_PROGRAM': 'inference.py'} 
        }
    ],
    ExecutionRoleArn=role,
)
print("Model Arn: " + create_model_response["ModelArn"])

Model name: sklearn-test2025-01-29-06-32-28
Model Arn: arn:aws:sagemaker:us-east-1:767397996001:model/sklearn-test2025-01-29-06-32-28


In [6]:

s3_client = boto3.client('s3')

# Specify the S3 bucket name and file path (evaluation metrics JSON file)
bucket_name = default_bucket  # Use the default bucket or replace with your bucket name
file_key = 'evaluation_metrics.json'  # The path to the evaluation metrics file

# Fetch the file from S3
response = s3_client.get_object(Bucket=bucket_name, Key=file_key)

# Read the JSON content from the file
json_data = response['Body'].read().decode('utf-8')  # Decode the content to text
data = json.loads(json_data)  # Load the JSON data

# Display the data in a pretty format
print(json.dumps(data, indent=4)) 

{
    "binary_classification_metrics": {
        "accuracy": {
            "value": 0.71,
            "standard_deviation": 0.0
        },
        "precision": {
            "value": 0.6969291851851852,
            "standard_deviation": 0.0
        },
        "recall": {
            "value": 0.71,
            "standard_deviation": 0.0
        }
    }
}


In [7]:
# Step 3: Register the model in Model Registry
model_package_group_name = "MLOps-AWS"  # Specify the model package group name
existing_model_package_groups = client.list_model_package_groups()["ModelPackageGroupSummaryList"]
group_exists = any(group["ModelPackageGroupName"] == model_package_group_name for group in existing_model_package_groups)


# Create a new Model Package Group if it doesn't exist
if not group_exists:
    try:
        create_model_package_group_response = client.create_model_package_group(
            ModelPackageGroupName=model_package_group_name,
            ModelPackageGroupDescription="Model Registry for the Zenon ML Ops Initiative." 
        )
        print("Model Package Group Arn: " + create_model_package_group_response["ModelPackageGroupArn"])
    except client.exceptions.ResourceInUse:
        print(f"Model Package Group '{model_package_group_name}' already exists.")
else:
    print(f"Model Package Group '{model_package_group_name}' already exists.")


# Create Model Package (Register the model)
create_model_package_response = client.create_model_package(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageDescription="Description of your model",
    ModelApprovalStatus="Approved",  # Set this to "Approved" if you want automatic approval
    InferenceSpecification={
        "Containers": [
            {
                "Image": image_uri,
                "ModelDataUrl": model_artifacts,
                "Environment": {'SAGEMAKER_SUBMIT_DIRECTORY': model_artifacts,
                                'SAGEMAKER_PROGRAM': 'inference.py'}
            }
        ],
        "SupportedContentTypes": ["application/json"],
        "SupportedResponseMIMETypes": ["application/json"]
    },
    MetadataProperties={
        "GeneratedBy": "manual-training",
        "ProjectId": "ml-ops-project-001" 
    },
    ModelMetrics={
        "ModelQuality": {
            "Statistics": {
                "ContentType": "application/json",
                "S3Uri": evaluation_metrics_uri 
            }
        }
    },
    CertifyForMarketplace=False
)
print("Model Package Arn: " + create_model_package_response["ModelPackageArn"])


Model Package Group Arn: arn:aws:sagemaker:us-east-1:767397996001:model-package-group/MLOps-AWS
Model Package Arn: arn:aws:sagemaker:us-east-1:767397996001:model-package/MLOps-AWS/1


In [8]:
#Step 2: EPC Creation
sklearn_epc_name = "sklearn-epc" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=sklearn_epc_name,
    ProductionVariants=[
        {
            "VariantName": "sklearnvariant",
            "ModelName": model_name,
            "InstanceType": "ml.t2.medium",
            "InitialInstanceCount": 1
        },
    ],
)
print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:767397996001:endpoint-config/sklearn-epc2025-01-29-06-32-40


In [9]:
#Step 3: EP Creation
endpoint_name = "sklearn-local-ep" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=sklearn_epc_name,
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])


#Monitor creation
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)
print(describe_endpoint_response)

Endpoint Arn: arn:aws:sagemaker:us-east-1:767397996001:endpoint/sklearn-local-ep2025-01-29-06-32-42
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService
{'EndpointName': 'sklearn-local-ep2025-01-29-06-32-42', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:767397996001:endpoint/sklearn-local-ep2025-01-29-06-32-42', 'EndpointConfigName': 'sklearn-epc2025-01-29-06-32-40', 'ProductionVariants': [{'VariantName': 'sklearnvariant', 'DeployedImages': [{'SpecifiedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3', 'ResolvedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn@sha256:1743f4967231a3e2a4cbd2dcd28bd5cac8647fad140d11f483286d592706939f', 'ResolutionTime': datetime.datetime(2025, 1, 29, 6, 32, 44, 207000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0